# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import csv  # Import the csv library for reading and writing CSV files
import numpy as np
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
vacation_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,9.45,98,100,3.58,NZ,1712806996
1,1,plastun,44.7561,136.2939,12.81,37,99,3.53,RU,1712807166
2,2,edinburgh of the seven seas,-37.0676,-12.3116,17.84,80,100,1.40,SH,1712806958
3,3,uturoa,-16.7333,-151.4333,29.26,76,20,8.92,PF,1712807168
4,4,adamstown,-25.0660,-130.1015,21.43,83,100,8.11,PN,1712806958


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Define a color palette for each city based on a categorical column (e.g., 'City')
map_plot_humidity = vacation_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    color='City',  # Assign a unique color to each city
    cmap='Category10',  # Use a categorical color palette
    legend='top_right',  # Position the legend to the top left
)

# Display the map plot
map_plot_humidity





:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
ideal_weather_df = vacation_df.loc[(vacation_df['Max Temp'] < 27) & (vacation_df['Max Temp'] > 21)]
ideal_weather_df






,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,adamstown,-25.0660,-130.1015,21.43,83,100,8.11,PN,1712806958
5,5,el porvenir,13.7617,-87.3458,26.63,78,60,0.85,HN,1712807171
18,18,bubaque,11.2833,-15.8333,23.77,73,0,5.67,GW,1712806953
21,21,mejit,10.2753,170.8646,26.64,76,100,9.83,MH,1712807022
28,28,lihue,21.9789,-159.3672,26.58,79,100,5.66,US,1712807077
...,...,...,...,...,...,...,...,...,...,...
556,556,pagar alam,-4.0167,103.2667,25.68,83,100,1.71,ID,1712807823
558,558,formosa,-26.1775,-58.1781,22.76,99,100,0.89,AR,1712807532
561,561,omdurman,15.6445,32.4777,24.96,14,2,5.60,SD,1712807829
563,563,naze,28.3667,129.4833,21.51,61,50,2.22,JP,1712807831


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:

# Create a new DataFrame called hotel_df
hotel_df = vacation_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,invercargill,NZ,-46.4000,168.3500,98,
1,plastun,RU,44.7561,136.2939,37,
2,edinburgh of the seven seas,SH,-37.0676,-12.3116,80,
3,uturoa,PF,-16.7333,-151.4333,76,
4,adamstown,PN,-25.0660,-130.1015,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000  # Define the radius for the hotel search
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1  # Limit to 1 result per city
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL for the API request
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    # print(latitude, longitude, name_address.url)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated DataFrame with hotel information
print(hotel_df)



Starting hotel search
invercargill - nearest hotel: The Grand
plastun - nearest hotel: Россияночка
edinburgh of the seven seas - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
adamstown - nearest hotel: No hotel found
el porvenir - nearest hotel: No hotel found
cole harbour - nearest hotel: Hearthstone Inn Boutique Hotel Dartmouth/Halifax
isola di capo rizzuto - nearest hotel: No hotel found
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
nyurba - nearest hotel: Марина
qaqortoq - nearest hotel: Hotel Qaqortoq
antofagasta - nearest hotel: Hotel Colón
punta arenas - nearest hotel: Hotel Lacolet
kodiak - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
udachny - nearest hotel: Вилюй
kavieng - nearest hotel: Kavieng Hotel
hermanus - nearest hotel: Aloe guest house
bubaque - nearest hotel: Chez Julio
pampa del infierno - nearest hotel: Biooceanico
aasiaat - nearest hotel: SØMA
mejit - nearest hotel: No hotel found
nemuro - nearest ho

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
map_plot_humidity = vacation_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    color='City',
    cmap='Category10',
    legend='top_right',
    labels='Hotel Name',  # Display 'Hotel Name' as labels on the plot points
)

map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)